In [1]:

pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install pandas-datareader

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [12]:

pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [13]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.arrays.sparse import dtype
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , LSTM

In [12]:
#LOAD DATA

company = ' RELIANCE.NS '

start = dt.datetime(2012,1,1)
end  = dt.datetime(2022,10,9)

data = web.DataReader(company,'yahoo',start,end)

TypeError: string indices must be integers

In [13]:
#preparing Data

scaler = MinMaxScaler(feature_range=(0,1))
#closing value
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))
#no of days to look into the past
prediction_days = 60

x_train = []
y_train = []
for x in range(prediction_days,len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x,0])
    y_train.append(scaled_data[x,0])
x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))


In [14]:
#BUILDING THE MODEL

model = Sequential()
model.add(LSTM(units = 50,return_sequences = True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units=1))#closing value

model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs= 25,batch_size=32)


Epoch 1/25
82/82 [==============================] - 7s 38ms/step - loss: 0.0114
Epoch 2/25
82/82 [==============================] - 3s 38ms/step - loss: 0.0036
Epoch 3/25
82/82 [==============================] - 3s 38ms/step - loss: 0.0034
Epoch 4/25
82/82 [==============================] - 3s 38ms/step - loss: 0.0029
Epoch 5/25
82/82 [==============================] - 3s 38ms/step - loss: 0.0029
Epoch 6/25
82/82 [==============================] - 3s 39ms/step - loss: 0.0024
Epoch 7/25
82/82 [==============================] - 3s 38ms/step - loss: 0.0023
Epoch 8/25
82/82 [==============================] - 3s 40ms/step - loss: 0.0025
Epoch 9/25
82/82 [==============================] - 3s 42ms/step - loss: 0.0022
Epoch 10/25
82/82 [==============================] - 4s 44ms/step - loss: 0.0020
Epoch 11/25
82/82 [==============================] - 4s 46ms/step - loss: 0.0020
Epoch 12/25
82/82 [==============================] - 3s 41ms/step - loss: 0.0026
Epoch 13/25
82/82 [==================

In [3]:
'''tesing model accuracy'''

#test data(currnet date)




test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()
test_data = web.DataReader(company,'yahoo',test_start,test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'],test_data['Close']),axis = 0)
model_inputs = total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)

#making predictions

x_test=[]
for x in range(prediction_days,len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])
x_test=np.array(x_test)
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

predicted_prices =model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

#plotting test predictions
plt.style.use('dark_background')
plt.plot(actual_prices,color= "black",label=f"actual {company} Price")
plt.plot(predicted_prices,color="green",label=f"predicted {company} Price")
plt.title(f"{company} Share Price",color="red")
plt.xlabel('Time',color="red")
plt.ylabel(f'{company} Share Price',color="red")
plt.legend()
plt.show()


NameError: name 'company' is not defined

In [16]:
#predicting next day

real_data = [model_inputs[len(model_inputs)+1 - prediction_days:len(model_inputs+1),0]]
real_data = np.array(real_data)
real_data =np.reshape(real_data,(real_data.shape[0],real_data.shape[1],1))
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

print(f"Prediction : {prediction}")

1/1 [==============================] - 1s 738ms/step
Prediction : [[1892.5605]]


In [ ]:
#sykit